In [5]:
import sys
from pathlib import Path

# Ensure we can import from src/
project_root = Path().resolve().parent
sys.path.append(str(project_root))

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from src.utils import RESULTS_DIR





In [6]:
data_file = RESULTS_DIR / "cleaned_data.csv"
df = pd.read_csv(data_file)

print("Loaded preprocessed data. Shape:", df.shape)
df.head()




Loaded preprocessed data. Shape: (2651306, 2)


,id,text
0,0,The presence of communication amid scientific ...
1,1,The Manhattan Project and its atomic bomb help...
2,2,Essay on The Manhattan Project - The Manhattan...
3,3,The Manhattan Project was the name for a proje...
4,4,versions of each volume as well as complementa...


In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load model directly on GPU
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda")

df = pd.read_csv("../data/msmarco/documents.csv")
sentences = df["text"].fillna("").astype(str).tolist()

embeddings = model.encode(
    sentences,
    batch_size=512, 
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Shape:", embeddings.shape)



Batches:   0%|          | 0/5179 [00:00<?, ?it/s]

Shape: (2651306, 384)


In [5]:
import faiss
import numpy as np
from pathlib import Path

RESULTS_DIR = Path("../results")

# Load existing embeddings
embeddings = np.load(RESULTS_DIR / "doc_embeddings.npy", mmap_mode='r')
dimension = embeddings.shape[1]

# Create IVF index
nlist = 100  # number of clusters
quantizer = faiss.IndexFlatL2(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

# Train on subset
train_size = min(10000, len(embeddings))
index.train(embeddings[:train_size])

# Add embeddings in chunks
chunk_size = 10000
for i in range(0, len(embeddings), chunk_size):
    index.add(embeddings[i:i+chunk_size])

# Save IVF index to disk
faiss.write_index(index, str(RESULTS_DIR / "faiss_index_ivf.bin"))




In [4]:

df.to_csv(RESULTS_DIR / "retriever_documents.csv", index=False)
print("Document metadata saved to results/retriever_documents.csv")


Document metadata saved to results/retriever_documents.csv
